In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import string

import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = '/home/manikya_varshney/Documents/Python/Yale/All Graphs/Test Data/final_processed_april01.csv'
data = pd.read_csv(path, low_memory=False)

In [3]:
keywords = ['bored' , 'lonely', 'stress', 
            'anxiety', 'scared', 'worry', 'end', 'cabin fever',
            '#sideeffectsofquarantinelife', 'tissue paper', 'toilet paper']

##### 1. Casing (Upper or lower case)
##### 2. Noise Removal (Removal of punctuation, white spaces, special characters, HTML tags)
##### 3. Tokenization (Tweets to tokens i.e. words seprated by spaces)
##### 4. Stopword Removal
##### 5. Text Normalization (Stemming and Lemmatization)

In [4]:
#Convert to lower
for i in range(len(keywords)): 
    keywords[i] = keywords[i].lower()

#Remove punctuations   
for i in range(len(keywords)): 
    keywords[i] = keywords[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords)): 
    keywords[i] = keywords[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_tokens = [sub.split() for sub in keywords] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_filtered=remove_stopwords(keywords)

#Stemming
ps = PorterStemmer()
keywords_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_filtered]
keywords_stem = [" ".join(sentence) for sentence in keywords_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_filtered]
keywords_final = [" ".join(sentence) for sentence in keywords_lem]

## Fuzzy Matching

In [5]:
data['final'] = data['final'].apply(str)
choices = data['final'].tolist()

In [6]:
 def fuzzy_m(row):
        keyword_match, score = process.extractOne(row['final'], keywords_final, scorer = fuzz.partial_ratio)
        row['final_score'] = score
        row['final_keyword_match'] = keyword_match
        return row

In [7]:
nan_value = float("NaN")
data['final'].replace("", nan_value, inplace=True)
data.dropna(subset = ["final"], inplace=True)

In [8]:
interim_ne = data.apply(fuzzy_m, axis=1)

In [9]:
interim_ne

,user_id,text_duplicate,final,final_score,final_keyword_match
0,2.843399e+07,As Chicago households fill out 2020 census dur...,chicago household fill out 2020 census coronav...,75,bore
1,1.010439e+08,Most of the press corps is very busy covering ...,the press corp busy cover the really important...,50,bore
2,1.647527e+07,@realDonaldTrump Mar 10–he was very specific: ...,mar 10–he specific google 1700 engineer work ...,100,end
3,1.647527e+07,"@realDonaldTrump , You were caustic &amp; sarc...",caustic amp sarcastic the rollout come even f...,75,scar
4,3.453977e+08,"Florida nears 8,000 coronavirus cases, as stat...",florida nears 8000 coronavirus case state repo...,67,end
...,...,...,...,...,...
16920,9.832288e+17,#SierraLeone has registered its index case of ...,sierraleone register index case covid19 from d...,67,lonely
16921,1.856380e+07,KTAR News reporter @TaylorKinnerup gives a cor...,ktar news reporter give coronavirus update ari...,67,stress
16922,9.186279e+08,Part of a 1938 public health map: https://t.co...,part 1938 public health map,45,toilet paper
16923,1.240031e+18,While the resourcefulness and hustle shown by ...,the resourcefulness hustle show many to produc...,67,stress


In [10]:
denominator = interim_ne.shape[0]

In [11]:
interim_ne['final_score'] = interim_ne['final_score'].astype(int)

In [12]:
interim_ne = interim_ne[interim_ne['final_score'] == 100]

In [13]:
interim_ne

,user_id,text_duplicate,final,final_score,final_keyword_match
2,16475267.0,@realDonaldTrump Mar 10–he was very specific: ...,mar 10–he specific google 1700 engineer work ...,100,end
10,394499101.0,AOC slams Cuomo's decision to suspend New York...,aoc slam cuomos decision to suspend new yorker...,100,end
18,95027069.0,How are we at #jpcarpet spending our #quaranti...,at jpcarpet spending quarantine building close...,100,end
41,15201225.0,"NEW BLOG! ""10 Self-Care Tips during a Stay-At-...",new blog 10 selfcare tip stayathome order frie...,100,end
92,7919632.0,This column by @karaswisher is the column my n...,column the column niece boyfriend need to read...,100,end
...,...,...,...,...,...
16815,39344374.0,My good friend &amp; hunting buddy @SteveDaine...,good friend amp hunt buddy need help midnight ...,100,end
16849,16614351.0,My wife is making masks but in true Jane fashi...,wife make mask true jane fashion make stylish ...,100,end
16860,612473.0,"James Corden says he's been struggling with ""i...",james corden say he struggle incredible spike ...,100,anxiety
16874,939091.0,Tonight marks our biggest public end-of-quarte...,tonight mark big public endofquarter deadline ...,100,end


In [14]:
numerator = interim_ne.shape[0]

In [15]:
proportion = (numerator/denominator)
print(proportion)

0.06091580502215657
